Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [2]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


In [3]:
def test_bond_energies(htf, is_old=True, is_solvated=False):
    # Get harmonic bond force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    harmonic_bond_force = system.getForce(0) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom bond force and hybrid positions
    bond_force_index = 1 if is_solvated else 0
    hybrid_system = htf.hybrid_system
    custom_bond_force = hybrid_system.getForce(bond_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_bond_force.getNumGlobalParameters()):
        if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_old':
            custom_bond_force.setGlobalParameterDefaultValue(i, lambda_old)
        if custom_bond_force.getGlobalParameterName(i) == 'lambda_0_bonds_new':
            custom_bond_force.setGlobalParameterDefaultValue(i, lambda_new)
            
#     for hybrid_idx, idx in htf._hybrid_to_environment_bond_indices.items():
#         p1, p2, length, k = harmonic_bond_force.getBondParameters(idx)
#         harmonic_bond_force.setBondParameters(hybrid_idx, p1, p2, length, k*0)
        
    # Zero the unique old/new bonds in the custom bond force
    hybrid_to_bond_indices = htf._hybrid_to_new_bond_indices if is_old else htf._hybrid_to_old_bond_indices
    for hybrid_idx, idx in hybrid_to_bond_indices.items():
        p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        index_to_zero = -3 if is_old else -1
        hybrid_params[index_to_zero] *= 0
        custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

#     # Zero the unique old/new bonds in the custom bond force
#     hybrid_to_bond_indices = htf._hybrid_to_environment_bond_indices
#     for hybrid_idx, idx in hybrid_to_bond_indices.items():
#         p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
#         hybrid_params = list(hybrid_params)
# #         index_to_zero = -1 if is_old else -3
#         hybrid_params[-1] *= 0
#         hybrid_params[-2] *= 0
    
            
#         custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)

    ## Get energy components of standard bond force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom bond force
    if is_solvated:
        custom_bond_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[0][1]], [components_hybrid[0][1]])
    
    print("Success! Custom bond force and standard bond force energies are equal!")

# Alanine dipeptide in vacuum

In [3]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.18434303979751374.
INFO:geometry:	reduced angle potential = 0.15002564669608512.
INFO:geometry:	reduced angle potential = 0.003907125640948157.
INFO:geometry:	reduced angle potential = 0.8178465245578943.
INFO:geometry:	reduced angle potential = 2.206305668455018.
INFO:geometry:	reduced angle potential = 0.004326388269150331.
INFO:geometry:	reduced angle potential = 0.20630702212828197.
INFO:geometry:	reduced angle potential = 0.034412963017395486.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angl

...

conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478861179005435e-06
INFO:geometry:Final logp_proposal: -26851.30047191144
INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_l

added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]
( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [5]:
with open("atp_vacuum.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [6]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [7]:
test_bond_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('CustomAngleForce', 13.805947690407493), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [8]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [9]:
test_bond_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.6906144830418987), ('HarmonicAngleForce', 13.351800265281843), ('PeriodicTorsionForce', 26.41852205265097), ('NonbondedForce', -54.709291264726836), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.6906144830418987), ('CustomAngleForce', 13.805947690407493), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [10]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.27934311243413673.
INFO:geometry:	reduced angle potential = 0.6870814800676404.


conducting subsequent work with the following platform: CUDA


INFO:geometry:	reduced angle potential = 0.004894353938349633.
INFO:geometry:	reduced angle potential = 0.05813624060687381.
INFO:geometry:	reduced angle potential = 0.2000535034679782.
INFO:geometry:	reduced angle potential = 0.4072513591942163.
INFO:geometry:	reduced angle potential = 0.8166026025320159.
INFO:geometry:	reduced angle potential = 0.3025760099105887.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined wi

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478861179005435e-06
INFO:geometry:Final logp_proposal: -26848.271139563196


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0)

( nonscale_lambda_bonds * nonscale_region+scale_lambda_0_bonds * scale_region_0+interscale_lambda_0_bonds * interscale_region_0 ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [11]:
with open("atp_vacuum_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [12]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [13]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]

In [14]:
test_bond_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('CustomAngleForce', 4.444211963504904), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [15]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [16]:
test_bond_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.344463940477031), ('HarmonicAngleForce', 3.990064538379252), ('PeriodicTorsionForce', 26.853516722533634), ('NonbondedForce', 148.62328263442106), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.3444639404770315), ('CustomAngleForce', 4.444211963504904), ('AndersenThermostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# Alanine dipeptide in solvent

In [17]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 19, 15, 13, 16, 17]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching fun

generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -27063.242955106478
INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...

added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:Generating new system exceptions dict...
INFO:relative:Handling constraints...
INFO:relative:Handling virtual sites...
INFO:relative:	_handle_virtual_sites: numVirtualSites: 0


( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [18]:
with open("atp_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [19]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [20]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -975.5666619711665), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('CustomAngleForce', 4.953858801999797), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [21]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [22]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 3.873550964508178), ('HarmonicAngleForce', 4.499711376874169), ('PeriodicTorsionForce', 27.57757321070606), ('NonbondedForce', -618.3003824188523), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.8735509645081847), ('CustomAngleForce', 4.953858801999797), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [23]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 13, 19, 15, 16, 17]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching fun

generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.9900002084555819, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 1]), 1: (19, 18, 10, 8, [1, Quantity(value=-0.07858019216845058, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 5]), 2: (19, 18, 10, 14, [1, Quantity(value=2.118586571250296, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 5]), 3: (19, 18, 10, 13, [1, Quantity(value=-2.149781203448759, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 5]), 4: (8, 10, 14, 17, [1, Quantity(value=-2.1249532223845904, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 8]), 5: (8, 10, 14, 16, [1, Quantity(value=2.0543043017157916, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 7]), 6: (8, 10, 14, 15, [1, Quantity(value=-0.02223392579985184, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 6]), 7: (18, 10, 14, 17, [1, Quantity(value=2.0255587336734213, unit=radian), Quantity(value=1200.0, u

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -27100.404073734037


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nano

( nonscale_lambda_bonds * nonscale_region+scale_lambda_0_bonds * scale_region_0+interscale_lambda_0_bonds * interscale_region_0 ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [24]:
with open("atp_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [25]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [26]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]

In [27]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -1773.618940558242), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03478365356521381), ('CustomAngleForce', 7.774941865660137), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [28]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [29]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.1778483298325095), ('HarmonicAngleForce', 7.320794440534433), ('PeriodicTorsionForce', 29.105734892070842), ('NonbondedForce', -1788.3329924439342), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1778483298325466), ('CustomAngleForce', 7.774941865660137), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# CDK2 transformation in solvent

In [30]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [32]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = None
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-05-11 11:31:42,780:(57.77s):root:	Detecting phases...
2021-05-11 11:31:42,784:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-05-11 11:31:42,788:(0.00s):root:No constraints will be removed
2021-05-11 11:31:42,791:(0.00s):root:No spectators
2021-05-11 11:31:42,794:(0.00s):root:			run_type is not specified; default to None
2021-05-11 11:31:42,798:(0.00s):root:	Detecting fe_type...
2021-05-11 11:31:42,801:(0.00s):root:		fe_type: repex
2021-05-11 11:31:42,804:(0.00s):root:			offline-freq not specified: default to 10.
2021-05-11 11:31:42,806:(0.00s):root:	'neglect_angles' detected: False.
2021-05-11 11:31:42,808:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-05-11 11:31:42,810:(0.00s):root:	Creating 'output'...
2021-05-11 11:31:42,825:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-05-11 11:31:42,826:(0.00s):r

2021-05-11 11:31:46,102:(0.00s):geometry:Conducting forward proposal...
2021-05-11 11:31:46,103:(0.00s):geometry:Computing proposal order with NetworkX...
2021-05-11 11:31:46,111:(0.01s):geometry:number of atoms to be placed: 1
2021-05-11 11:31:46,112:(0.00s):geometry:Atom index proposal order is [2190]
2021-05-11 11:31:46,114:(0.00s):geometry:omitted_bonds: []
2021-05-11 11:31:46,115:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-05-11 11:31:46,258:(0.14s):geometry:creating growth system...
2021-05-11 11:31:46,294:(0.04s):geometry:	creating bond force...
2021-05-11 11:31:46,295:(0.00s):geometry:	there are 27 bonds in reference force.
2021-05-11 11:31:46,297:(0.00s):geometry:	creating angle force...
2021-05-11 11:31:46,298:(0.00s):geometry:	there are 85 angles in reference force.
2021-05-11 11:31:46,300:(0.00s):geometry:	creating torsion force...
2021-05-11 11:31:46,302:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-05-11 11:31:47,689:(1.33s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-05-11 11:31:57,077:(1.46s):geometry:total reduced energy added from growth system: -5.056837038117532
2021-05-11 11:31:57,079:(0.00s):geometry:final reduced energy 43.36216011253515
2021-05-11 11:31:57,081:(0.00s):geometry:sum of energies: 43.36215903557945
2021-05-11 11:31:57,082:(0.00s):geometry:magnitude of difference in the energies: 1.0769557015066766e-06
2021-05-11 11:31:57,084:(0.00s):geometry:Final logp_proposal: 11.56594796641107
2021-05-11 11:31:57,188:(0.10s):root:	writing pickle output...


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423200139534058), ('CustomBondForce', -5.1962986311777435)]


2021-05-11 11:31:57,306:(0.12s):root:	successfully dumped pickle.
2021-05-11 11:31:57,310:(0.00s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-05-11 11:31:57,311:(0.00s):root:	writing atom_mapping.png
2021-05-11 11:31:58,284:(0.97s):root:	 steps per move application: 50
2021-05-11 11:31:58,287:(0.00s):root:	trajectory prefix: out
2021-05-11 11:31:58,290:(0.00s):root:	atom selection detected: not water
2021-05-11 11:31:58,292:(0.00s):root:	no nonequilibrium detected.
2021-05-11 11:31:58,294:(0.00s):root:	cataloging HybridTopologyFactories...
2021-05-11 11:31:58,296:(0.00s):root:		phase: solvent:
2021-05-11 11:31:58,299:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-05-11 11:31:58,301:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-05-11 11:31:58,303:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-05-11 11:31:58,354:(0.05s):relative:Old sy

( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [33]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [34]:
with open("cdk2_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [35]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [36]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1589.3730460447086), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.698600437266055), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [37]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [38]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.6522372518708), ('PeriodicTorsionForce', 15.080012167241955), ('NonbondedForce', 1608.908174275118), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899664), ('CustomAngleForce', 29.698600437266055), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [39]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

In [40]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = [[44, 2197]]
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-05-11 11:32:11,141:(12.65s):root:	Detecting phases...
2021-05-11 11:32:11,145:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-05-11 11:32:11,149:(0.00s):root:No constraints will be removed
2021-05-11 11:32:11,152:(0.00s):root:No spectators
2021-05-11 11:32:11,156:(0.00s):root:			run_type is not specified; default to None
2021-05-11 11:32:11,159:(0.00s):root:	Detecting fe_type...
2021-05-11 11:32:11,162:(0.00s):root:		fe_type: repex
2021-05-11 11:32:11,165:(0.00s):root:			offline-freq not specified: default to 10.
2021-05-11 11:32:11,168:(0.00s):root:	'neglect_angles' detected: False.
2021-05-11 11:32:11,169:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-05-11 11:32:11,170:(0.00s):root:	Creating 'output'...
2021-05-11 11:32:11,188:(0.02s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-05-11 11:32:11,189:(0.00s):r

2021-05-11 11:32:14,379:(0.00s):geometry:Conducting forward proposal...
2021-05-11 11:32:14,380:(0.00s):geometry:Computing proposal order with NetworkX...
2021-05-11 11:32:14,388:(0.01s):geometry:number of atoms to be placed: 1
2021-05-11 11:32:14,389:(0.00s):geometry:Atom index proposal order is [2190]
2021-05-11 11:32:14,391:(0.00s):geometry:omitted_bonds: []
2021-05-11 11:32:14,392:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-05-11 11:32:14,534:(0.14s):geometry:creating growth system...
2021-05-11 11:32:14,570:(0.04s):geometry:	creating bond force...
2021-05-11 11:32:14,572:(0.00s):geometry:	there are 27 bonds in reference force.
2021-05-11 11:32:14,573:(0.00s):geometry:	creating angle force...
2021-05-11 11:32:14,575:(0.00s):geometry:	there are 85 angles in reference force.
2021-05-11 11:32:14,577:(0.00s):geometry:	creating torsion force...
2021-05-11 11:32:14,578:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-05-11 11:32:15,086:(0.45s):geometry:setting atoms_with_positions context new positions
2021-05-11 11:32:15,374:(0.29s):geometry:There are 1 new atoms
2021-05-11 11:32:15,381:(0.01s):geometry:	reduced angle potential = 0.46488727378492417.


conducting subsequent work with the following platform: CUDA


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-05-11 11:32:20,068:(0.89s):geometry:total reduced energy added from growth system: -5.056837038117532
2021-05-11 11:32:20,071:(0.00s):geometry:final reduced energy 43.36216011253515
2021-05-11 11:32:20,072:(0.00s):geometry:sum of energies: 43.36215903557945
2021-05-11 11:32:20,073:(0.00s):geometry:magnitude of difference in the energies: 1.0769557015066766e-06
2021-05-11 11:32:20,075:(0.00s):geometry:Final logp_proposal: 11.810926160630629


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423200139534058), ('CustomBondForce', -5.1962986311777435)]


2021-05-11 11:32:20,219:(0.14s):root:	writing pickle output...
2021-05-11 11:32:20,336:(0.12s):root:	successfully dumped pickle.
2021-05-11 11:32:20,339:(0.00s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-05-11 11:32:20,341:(0.00s):root:	writing atom_mapping.png
2021-05-11 11:32:20,386:(0.04s):root:	 steps per move application: 50
2021-05-11 11:32:20,387:(0.00s):root:	trajectory prefix: out
2021-05-11 11:32:20,389:(0.00s):root:	atom selection detected: not water
2021-05-11 11:32:20,390:(0.00s):root:	no nonequilibrium detected.
2021-05-11 11:32:20,391:(0.00s):root:	cataloging HybridTopologyFactories...
2021-05-11 11:32:20,393:(0.00s):root:		phase: solvent:
2021-05-11 11:32:20,394:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-05-11 11:32:20,396:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-05-11 11:32:20,397:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptio

( nonscale_lambda_bonds * nonscale_region+scale_lambda_0_bonds * scale_region_0+interscale_lambda_0_bonds * interscale_region_0 ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [58]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [41]:
with open("cdk2_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [42]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [43]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[44, 2197]

In [44]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1589.3730460447086), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.698600437266055), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [45]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [46]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.6522372518708), ('PeriodicTorsionForce', 15.080012167241955), ('NonbondedForce', 1608.908174275118), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.519508183899664), ('CustomAngleForce', 29.698600437266055), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


# 8mer in solvent

In [1]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/rjmc/coordinate_numba.py:14: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  n_2 = np.dot(a, a)
/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/rjmc/coordinate_numba.py:65: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  d_ang = np.dot(angle_rotation_matrix, d_r)
/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/rjmc/coordinate_numba.py:71: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  d_torsion = np.dot(torsion_rotation_matrix, d_ang)
/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.

conducting subsequent work with the following platform: CUDA


In [2]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 0.00025,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [3]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
# aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
# mutant = random.choice(aminos_updated)
mutant = "MET"
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-05-11 13:49:23,411:(0.00s):proposal_generator:	Conducting polymer point mutation proposal...


Making mutation ALA->MET


2021-05-11 13:49:23,760:(0.35s):proposal_generator:Using matching_criterion to chose best atom map
2021-05-11 13:49:23,764:(0.00s):proposal_generator:Scaffold has symmetry of 0
2021-05-11 13:49:23,765:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-05-11 13:49:23,767:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-05-11 13:49:23,769:(0.00s):proposal_generator:len [{13: 8}, {16: 9}, {15: 9}, {14: 9}, {13: 9}, {12: 9}, {11: 9}, {10: 9}, {16: 8}, {15: 8}, {14: 8}, {10: 7}, {12: 8}, {11: 8}, {10: 8}, {16: 7}, {15: 7}, {14: 7}, {13: 7}, {12: 7}, {11: 7}]
2021-05-11 13:49:23,771:(0.00s):proposal_generator:{13: 8}
2021-05-11 13:49:23,772:(0.00s):proposal_generator:{16: 9}
2021-05-11 13:49:23,774:(0.00s):proposal_generator:{15: 9}
2021-05-11 13:49:23,775:(0.00s):proposal_generator:{14: 9}
2021-05-11 13:49:23,776:(0.00s):proposal_generator:{13: 9}
2021-05-11 13:49:

conducting subsequent work with the following platform: CUDA


2021-05-11 13:49:25,876:(0.35s):geometry:setting atoms_with_positions context new positions
2021-05-11 13:49:26,162:(0.29s):geometry:There are 11 new atoms
2021-05-11 13:49:26,169:(0.01s):geometry:	reduced angle potential = 0.0016279810856661186.


...

conducting subsequent work with the following platform: CUDA


2021-05-11 13:49:32,631:(0.76s):geometry:total reduced potential before atom placement: 168.36662826283057


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-05-11 13:49:33,460:(0.83s):geometry:total reduced energy added from growth system: 19.11335785388209
2021-05-11 13:49:33,462:(0.00s):geometry:final reduced energy 187.47998570143525
2021-05-11 13:49:33,464:(0.00s):geometry:sum of energies: 187.47998611671267
2021-05-11 13:49:33,465:(0.00s):geometry:magnitude of difference in the energies: 4.1527740890501263e-07
2021-05-11 13:49:33,467:(0.00s):geometry:Final logp_proposal: -30493.35014121708


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.04562463240715887), ('CustomAngleForce', 0.11963264577965825), ('CustomTorsionForce', 7.457414545448549), ('CustomBondForce', 11.490686030246726)]


In [4]:
from perses.annihilation.relative import RxnHybridTopologyFactory
flatten_exceptions = True
flatten_torsions = True
endstate = None
scale_regions = None
htf = RxnHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=positions,
                     new_positions=new_positions,
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=flatten_exceptions,
                     omitted_terms=None,
                     flatten_torsions=flatten_torsions,
                     scale_regions=scale_regions)

2021-05-11 13:49:33,564:(0.10s):relative:*** Generating RxnHybridTopologyFactory ***
2021-05-11 13:49:33,566:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-05-11 13:49:33,668:(0.10s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-05-11 13:49:33,669:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-05-11 13:49:33,671:(0.00s):relative:No unknown forces.
2021-05-11 13:49:33,672:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-05-11 13:49:33,674:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-05-11 13:49:33,715:(0.04s):relative:Adding and mapping new atoms to hybrid system...
2021-05-11 13:49:33,716:(0.00s):relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']

2021-05-11 13:49:34,060:(0.00s):relative:hybrid index pair: (44, 47, 48, 55), terms: [[158, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.66944, unit=kilojoule/mole)]], new term collector key: (44, 47, 48, 55), term: [[185, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.66944, unit=kilojoule/mole)]]
2021-05-11 13:49:34,062:(0.00s):relative:hybrid index pair: (45, 44, 47, 48), terms: [[159, 4, Quantity(value=0.0, unit=radian), Quantity(value=0.79496, unit=kilojoule/mole)], [160, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.602496, unit=kilojoule/mole)], [161, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=2.59408, unit=kilojoule/mole)], [162, 1, Quantity(value=0.0, unit=radian), Quantity(value=2.953904, unit=kilojoule/mole)]], new term collector key: (45, 44, 47, 48), term: [[186, 4, Quantity(value=0.0, unit=radian), Quantity(value=0.79496, unit=kilojoule/mole)], [187, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.602496, unit=kilojoule

2021-05-11 13:49:34,093:(0.00s):relative:hybrid index pair: (49, 48, 50, 60), terms: [[188, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.66944, unit=kilojoule/mole)]], new term collector key: (49, 48, 50, 60), term: [[215, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.66944, unit=kilojoule/mole)]]
2021-05-11 13:49:34,095:(0.00s):relative:hybrid index pair: (49, 48, 50, 61), terms: [[189, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.66944, unit=kilojoule/mole)]], new term collector key: (49, 48, 50, 61), term: [[216, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.66944, unit=kilojoule/mole)]]
2021-05-11 13:49:34,096:(0.00s):relative:hybrid index pair: (50, 48, 47, 53), terms: [[190, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.66944, unit=kilojoule/mole)]], new term collector key: (50, 48, 47, 53), term: [[217, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.66944, unit=kilojoule/mole)]]
2021-05-11 13:49:34,097:(0.00s):relative:hybrid inde

2021-05-11 13:49:34,121:(0.00s):relative:hybrid index pair: (63, 66, 67, 68), terms: [[216, 4, Quantity(value=0.0, unit=radian), Quantity(value=0.577392, unit=kilojoule/mole)], [217, 3, Quantity(value=3.141592653589793, unit=radian), Quantity(value=1.723808, unit=kilojoule/mole)], [218, 2, Quantity(value=0.0, unit=radian), Quantity(value=0.347272, unit=kilojoule/mole)], [219, 1, Quantity(value=3.141592653589793, unit=radian), Quantity(value=0.820064, unit=kilojoule/mole)]], new term collector key: (63, 66, 67, 68), term: [[243, 4, Quantity(value=0.0, unit=radian), Quantity(value=0.577392, unit=kilojoule/mole)], [244, 3, Quantity(value=3.141592653589793, unit=radian), Quantity(value=1.723808, unit=kilojoule/mole)], [245, 2, Quantity(value=0.0, unit=radian), Quantity(value=0.347272, unit=kilojoule/mole)], [246, 1, Quantity(value=3.141592653589793, unit=radian), Quantity(value=0.820064, unit=kilojoule/mole)]]
2021-05-11 13:49:34,132:(0.01s):relative:hybrid index pair: (63, 66, 67, 75), te

2021-05-11 13:49:34,145:(0.00s):relative:hybrid index pair: (68, 67, 66, 73), terms: [[248, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.6508444444444444, unit=kilojoule/mole)]], new term collector key: (68, 67, 66, 73), term: [[275, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.6508444444444444, unit=kilojoule/mole)]]
2021-05-11 13:49:34,146:(0.00s):relative:hybrid index pair: (68, 67, 66, 74), terms: [[249, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.6508444444444444, unit=kilojoule/mole)]], new term collector key: (68, 67, 66, 74), term: [[276, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.6508444444444444, unit=kilojoule/mole)]]
2021-05-11 13:49:34,147:(0.00s):relative:hybrid index pair: (70, 68, 67, 75), terms: [[250, 3, Quantity(value=3.141592653589793, unit=radian), Quantity(value=0.33472, unit=kilojoule/mole)], [251, 1, Quantity(value=0.0, unit=radian), Quantity(value=3.3472000000000004, unit=kilojoule/mole)]], new term collector key: (70, 68,

...

2021-05-11 13:49:34,190:(0.00s):relative:hybrid index pair: (93, 92, 91, 95), terms: [[310, 1, Quantity(value=0.0, unit=radian), Quantity(value=3.3472000000000004, unit=kilojoule/mole)], [311, 3, Quantity(value=3.141592653589793, unit=radian), Quantity(value=0.33472, unit=kilojoule/mole)]], new term collector key: (93, 92, 91, 95), term: [[337, 1, Quantity(value=0.0, unit=radian), Quantity(value=3.3472000000000004, unit=kilojoule/mole)], [338, 3, Quantity(value=3.141592653589793, unit=radian), Quantity(value=0.33472, unit=kilojoule/mole)]]
2021-05-11 13:49:34,191:(0.00s):relative:hybrid index pair: (93, 92, 91, 96), terms: [[312, 1, Quantity(value=0.0, unit=radian), Quantity(value=3.3472000000000004, unit=kilojoule/mole)], [313, 3, Quantity(value=3.141592653589793, unit=radian), Quantity(value=0.33472, unit=kilojoule/mole)]], new term collector key: (93, 92, 91, 96), term: [[339, 1, Quantity(value=0.0, unit=radian), Quantity(value=3.3472000000000004, unit=kilojoule/mole)], [340, 3, Qua

( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


2021-05-11 13:49:34,217:(0.00s):relative:hybrid index pair: (100, 99, 117, 118), terms: [[340, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=10.46, unit=kilojoule/mole)]], new term collector key: (100, 99, 117, 118), term: [[367, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=10.46, unit=kilojoule/mole)]]
2021-05-11 13:49:34,218:(0.00s):relative:hybrid index pair: (100, 99, 117, 119), terms: [[341, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=10.46, unit=kilojoule/mole)], [342, 1, Quantity(value=0.0, unit=radian), Quantity(value=8.368, unit=kilojoule/mole)]], new term collector key: (100, 99, 117, 119), term: [[368, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=10.46, unit=kilojoule/mole)], [369, 1, Quantity(value=0.0, unit=radian), Quantity(value=8.368, unit=kilojoule/mole)]]
2021-05-11 13:49:34,219:(0.00s):relative:hybrid index pair: (101, 98, 99, 117), terms: [[343, 3, Quantity(value=0.0, unit=radian), Q

2021-05-11 13:49:34,242:(0.00s):relative:hybrid index pair: (107, 105, 103, 112), terms: [[365, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=15.167, unit=kilojoule/mole)]], new term collector key: (107, 105, 103, 112), term: [[392, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=15.167, unit=kilojoule/mole)]]
2021-05-11 13:49:34,242:(0.00s):relative:hybrid index pair: (107, 106, 104, 113), terms: [[366, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=15.167, unit=kilojoule/mole)]], new term collector key: (107, 106, 104, 113), term: [[393, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=15.167, unit=kilojoule/mole)]]
2021-05-11 13:49:34,243:(0.00s):relative:hybrid index pair: (109, 98, 101, 110), terms: [[367, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.6508444444444444, unit=kilojoule/mole)]], new term collector key: (109, 98, 101, 110), term: [[394, 3, Quantity(value=0.0, unit=radian), Quantity(value

2021-05-11 13:49:34,269:(0.00s):relative:hybrid index pair: (105, 106, 107, 116), terms: [[397, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=4.6024, unit=kilojoule/mole)]], new term collector key: (106, 105, 107, 116), term: [[424, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=4.6024, unit=kilojoule/mole)]]
2021-05-11 13:49:34,271:(0.00s):relative:order of atoms does not match
2021-05-11 13:49:34,271:(0.00s):relative:hybrid index pair: (99, 118, 117, 119), terms: [[398, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=4.6024, unit=kilojoule/mole)]], new term collector key: (99, 118, 117, 119), term: [[425, 2, Quantity(value=3.141592653589793, unit=radian), Quantity(value=4.6024, unit=kilojoule/mole)]]


In [ ]:
# ( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;

# K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 
# + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;

# length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 
# + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [5]:
# Save htf
# with open("8mer_vacuum.pickle", "wb") as f:
#     pickle.dump(htf, f)
with open("8mer_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [6]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [7]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2abb8b6c8ab0> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2abb8b6c8bd0> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2abb8b6c8e40> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2abb8b6c8d80> >]

In [8]:
custom_bond_force = htf.hybrid_system.getForce(3)

In [9]:
for i in htf._hybrid_to_environment_torsion_indices:
    p1, p2, p3, p4, hybrid_params = custom_bond_force.getTorsionParameters(i)
    print(p1, p2, p3, p4, hybrid_params)

2 1 0 3 (1.0, 1.0, 0.0, 1.0, 0.0, 3.3472000000000004, 1.0, 0.0, 0.0)
2 1 0 3 (1.0, 1.0, 0.0, 3.0, 3.141592653589793, 0.33472, 3.0, 3.141592653589793, 0.0)
2 1 0 4 (1.0, 1.0, 0.0, 1.0, 0.0, 3.3472000000000004, 1.0, 0.0, 0.0)
2 1 0 4 (1.0, 1.0, 0.0, 3.0, 3.141592653589793, 0.33472, 3.0, 3.141592653589793, 0.0)
2 1 0 5 (1.0, 1.0, 0.0, 1.0, 0.0, 3.3472000000000004, 1.0, 0.0, 0.0)
2 1 0 5 (1.0, 1.0, 0.0, 3.0, 3.141592653589793, 0.33472, 3.0, 3.141592653589793, 0.0)
43 44 45 62 (1.0, 1.0, 0.0, 3.0, 3.141592653589793, 2.3012, 3.0, 3.141592653589793, 0.0)
43 44 45 62 (1.0, 1.0, 0.0, 2.0, 3.141592653589793, 6.610720000000001, 2.0, 3.141592653589793, 0.0)
43 44 45 62 (1.0, 1.0, 0.0, 1.0, 3.141592653589793, 1.8828, 1.0, 3.141592653589793, 0.0)
43 44 47 48 (1.0, 1.0, 0.0, 4.0, 0.0, 0.305432, 4.0, 0.0, 0.0)
43 44 47 48 (1.0, 1.0, 0.0, 3.0, 0.0, 0.602496, 3.0, 0.0, 0.0)
43 44 47 48 (1.0, 1.0, 0.0, 2.0, 3.141592653589793, 1.0836560000000002, 2.0, 3.141592653589793, 0.0)
43 44 47 48 (1.0, 1.0, 0.0, 1.

In [53]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', -10389.987370171535), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153174), ('CustomAngleForce', 304.7317311797578), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [54]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [55]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.2544807917460155), ('HarmonicAngleForce', 304.4890027961487), ('PeriodicTorsionForce', 151.33915324689795), ('NonbondedForce', -10142.071695388939), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.2544807917460155), ('CustomAngleForce', 304.7317311797578), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


### Test one alchemical region, one scale region

In [56]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

In [57]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 0.00025,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [58]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
mutant = random.choice(aminos_updated)
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-05-11 11:36:15,129:(172.01s):proposal_generator:	Conducting polymer point mutation proposal...


Making mutation ALA->GLY


2021-05-11 11:36:15,408:(0.28s):proposal_generator:Using matching_criterion to chose best atom map
2021-05-11 11:36:15,411:(0.00s):proposal_generator:Scaffold has symmetry of 0
2021-05-11 11:36:15,412:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-05-11 11:36:15,413:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-05-11 11:36:15,415:(0.00s):proposal_generator:len [{5: 6}, {6: 6}]
2021-05-11 11:36:15,417:(0.00s):proposal_generator:{5: 6}
2021-05-11 11:36:15,418:(0.00s):proposal_generator:{6: 6}
2021-05-11 11:36:15,419:(0.00s):proposal_generator:Returning map that best satisfies matching_criterion
2021-05-11 11:36:15,421:(0.00s):proposal_generator:Finding best map using matching_criterion name
2021-05-11 11:36:15,422:(0.00s):proposal_generator:{5: 6}
2021-05-11 11:36:16,303:(0.88s):geometry:propose: performing forward proposal
2021-05-11 11:36:16,305:(0.00s)

conducting subsequent work with the following platform: CUDA


2021-05-11 11:36:17,590:(0.45s):geometry:setting atoms_with_positions context new positions
2021-05-11 11:36:17,912:(0.32s):geometry:There are 2 new atoms
2021-05-11 11:36:17,923:(0.01s):geometry:	reduced angle potential = 1.5177149526591316.


...

conducting subsequent work with the following platform: CUDA


2021-05-11 11:36:23,689:(0.82s):geometry:total reduced potential before atom placement: 167.96576588447826


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-05-11 11:36:24,601:(0.91s):geometry:total reduced energy added from growth system: 18.779365142563123
2021-05-11 11:36:24,604:(0.00s):geometry:final reduced energy 186.74513005483178
2021-05-11 11:36:24,605:(0.00s):geometry:sum of energies: 186.7451310270414
2021-05-11 11:36:24,607:(0.00s):geometry:magnitude of difference in the energies: 9.722096052655615e-07
2021-05-11 11:36:24,608:(0.00s):geometry:Final logp_proposal: -27946.739550944832


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.04562463240715887), ('CustomAngleForce', 0.24272838360905494), ('CustomTorsionForce', 7.735181302225758), ('CustomBondForce', 10.755830824321151)]


In [60]:
from perses.annihilation.relative import RxnHybridTopologyFactory
flatten_exceptions = True
flatten_torsions = True
endstate = None
scale_regions = [list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])]
htf = RxnHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=positions,
                     new_positions=new_positions,
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=flatten_exceptions,
                     omitted_terms=None,
                     flatten_torsions=flatten_torsions,
                     scale_regions=scale_regions)

2021-05-11 11:36:53,775:(28.90s):relative:*** Generating RxnHybridTopologyFactory ***
2021-05-11 11:36:53,785:(0.01s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-05-11 11:36:53,946:(0.16s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-05-11 11:36:53,948:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-05-11 11:36:53,949:(0.00s):relative:No unknown forces.
2021-05-11 11:36:53,951:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-05-11 11:36:53,952:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-05-11 11:36:53,993:(0.04s):relative:Adding and mapping new atoms to hybrid system...
2021-05-11 11:36:53,994:(0.00s):relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', '

( nonscale_lambda_bonds * nonscale_region+scale_lambda_0_bonds * scale_region_0+interscale_lambda_0_bonds * interscale_region_0 ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [61]:
# Save htf
with open("8mer_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [62]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [63]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[21, 23, 24, 25, 26, 5576, 5575]

In [64]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.300105424153174), ('HarmonicAngleForce', 12.445685055931849), ('PeriodicTorsionForce', 156.24350906389375), ('NonbondedForce', -10187.074749800007), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.300105424153174), ('CustomAngleForce', 160.11384586447), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [65]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [66]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.2544807917460155), ('HarmonicAngleForce', 159.87111748086093), ('PeriodicTorsionForce', 151.16813048949732), ('NonbondedForce', -10055.098660066074), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.2544807917460155), ('CustomAngleForce', 160.11384586447), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


## Barnase:barstar (old htf, no naked charge fix or counterion fix)

### Test one alchemical region, no scale regions

In [17]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor

In [18]:
from __future__ import absolute_import

from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol, oechem
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory, RxnHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
    """
    Simple, stripped-down class to create a protein-ligand system and allow a mutation of a protein.
    this will allow support for the creation of _two_ relative free energy calculations:
        1. 'wildtype' - 'point mutant' complex hybrid.
        2. 'wildtype' - 'point mutant' protein hybrid (i.e. with ligand of interest unbound)
    Example (create full point mutation executor and run parallel tempering on both complex and apo phases):
        from pkg_resources import resource_filename
        protein_path = 'data/perses_jacs_systems/thrombin/Thrombin_protein.pdb'
        ligands_path = 'data/perses_jacs_systems/thrombin/Thrombin_ligands.sdf'
        protein_filename = resource_filename('openmmforcefields', protein_path)
        ligand_input = resource_filename('openmmforcefields', ligands_path)
        pm_delivery = PointMutationExecutor(protein_filename=protein_filename,
                                    mutation_chain_id='2',
                                    mutation_residue_id='198',
                                     proposed_residue='THR',
                                     phase='complex',
                                     conduct_endstate_validation=False,
                                     ligand_input=ligand_input,
                                     ligand_index=0,
                                     forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                                     barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                                     forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                                     small_molecule_forcefields='gaff-2.11')
        complex_htf = pm_delivery.get_complex_htf()
        apo_htf = pm_delivery.get_apo_htf()
        # Now we can build the hybrid repex samplers
        from perses.annihilation.lambda_protocol import LambdaProtocol
        from openmmtools.multistate import MultiStateReporter
        from perses.samplers.multistate import HybridRepexSampler
        from openmmtools import mcmc
        suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000
        for htf in [complex_htf, apo_htf]:
            lambda_protocol = LambdaProtocol(functions='default')
            reporter_file = 'reporter.nc'
            reporter = MultiStateReporter(reporter_file, analysis_particle_indices = htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
            hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                  collision_rate=5.0 / unit.picosecond,
                                                                                  n_steps=250,
                                                                                  reassign_velocities=False,
                                                                                  n_restart_attempts=20,
                                                                                  splitting="V R R R O R R R V",
                                                                                  constraint_tolerance=1e-06),
                                                                                  hybrid_factory=htf, online_analysis_interval=10)
            hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file=reporter, lambda_protocol=lambda_protocol, endstates=False)
            hss.extend(n_cycles)
    """
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 repartitioned_endstate=None,
                 rxn_field = True,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_file : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            repartitioned_endstate : int, default None
                the endstate (0 or 1) at which to build the RepartitionedHybridTopologyFactory. By default, this is None,
                meaning a vanilla HybridTopologyFactory will be built.
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_input, index=ligand_index)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for (top, pos, sys) in inputs:
            point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                                 system_generator=self.system_generator,
                                                                 chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                 max_point_mutants=1,
                                                                 residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                 allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                 aggregate=True) # Always allow aggregation

            topology_proposal = point_mutation_engine.propose(sys, top)

            # Only validate energy bookkeeping if the WT and proposed residues do not involve rings
            old_res = [res for res in top.residues() if res.id == mutation_residue_id][0]
            validate_bool = False if old_res.name in ring_amino_acids or proposed_residue in ring_amino_acids else True
            new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                                   validate_energy_bookkeeping=validate_bool)
            logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
                                                        validate_energy_bookkeeping=validate_bool)

            if rxn_field:
                factory = RxnHybridTopologyFactory
            else:
                if repartitioned_endstate is None:
                    factory = HybridTopologyFactory
                elif repartitioned_endstate in [0, 1]:
                    factory = RepartitionedHybridTopologyFactory

            forward_htf = factory(topology_proposal=topology_proposal,
                                  current_positions=pos,
                                  new_positions=new_positions,
                                  use_dispersion_correction=False,
                                  functions=None,
                                  softcore_alpha=None,
                                  bond_softening_constant=1.0,
                                  angle_softening_constant=1.0,
                                  soften_only_new=False,
                                  neglected_new_angle_terms=[],
                                  neglected_old_angle_terms=[],
                                  softcore_LJ_v2=True,
                                  softcore_electrostatics=True,
                                  softcore_LJ_v2_alpha=0.85,
                                  softcore_electrostatics_alpha=0.3,
                                  softcore_sigma_Q=1.0,
                                  interpolate_old_and_new_14s=flatten_exceptions,
                                  omitted_terms=None,
                                  endstate=repartitioned_endstate,
                                  flatten_torsions=flatten_torsions)

            if not topology_proposal.unique_new_atoms:
                assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
                assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            else:
                added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

            if not topology_proposal.unique_old_atoms:
                assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
                assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
                subtracted_valence_energy = 0.0
            else:
                subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


            if conduct_endstate_validation and repartitioned_endstate is None:
                zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta=beta, ENERGY_THRESHOLD=ENERGY_THRESHOLD)
                if zero_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 0 state is above the threshold ({ENERGY_THRESHOLD}): {zero_state_error}")
                if one_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 1 state is above the threshold ({ENERGY_THRESHOLD}): {one_state_error}")
            else:
                pass

            htfs.append(forward_htf)

        self.apo_htf = htfs[0]
        self.complex_htf = htfs[1] if ligand_input else None

In [19]:
solvent_delivery = PointMutationExecutor2("../../input/mmc2_barstar.pdb",
                        '1',
                        "42",
                        "ALA",
                        ligand_input="../../input/mmc2_barnase.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        conduct_endstate_validation=False,
                        rxn_field=True
                       )

INFO:root:solvating at 0.05 M using tip3p
INFO:root:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:{7: 11}
INFO:proposal_generator:{8: 11}
INFO:proposal_generator:{9: 11}
INFO:proposal_generator:{7: 12}
INFO:proposal_generator:{8: 12}
INFO:proposal_generator:{9: 12}
INFO:proposal_generator:{7: 13}
INFO:proposal_generator:{8: 13}
INFO:proposal_generator:{9: 13}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:pr

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 4 new atoms
INFO:geometry:	reduced angle potential = 0.4042126110219984.
INFO:geometry:	reduced angle potential = 1.5282998916822412.
INFO:geometry:	reduced angle potential = 0.0884430112660456.
INFO:geometry:	reduced angle potential = 0.09757108073714976.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 733 bond forces in the no-nonbonded final system
INFO:geometry:	there are 2628 angle forces in the no-nonbonded final system
INFO:geometry:	there are 4939 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

added energy components: [('CustomBondForce', 0.4083956553523612), ('CustomAngleForce', 1.622079514297996), ('CustomTorsionForce', 17.50313698245956), ('CustomBondForce', -81.92370902708515)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=V

( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


INFO:relative:hybrid_index_pair (700, 703, 702, 712) was not found in the new_term_collector, but (703, 700, 702, 712) has the same atoms and terms, so (703, 700, 702, 712) will be removed from the new term collector
INFO:relative:hybrid_index_pair (701, 702, 703, 713) was not found in the new_term_collector, but (702, 701, 703, 713) has the same atoms and terms, so (702, 701, 703, 713) will be removed from the new term collector
INFO:relative:hybrid_index_pair (750, 754, 752, 762) was not found in the new_term_collector, but (754, 750, 752, 762) has the same atoms and terms, so (754, 750, 752, 762) will be removed from the new term collector
INFO:relative:hybrid_index_pair (862, 859, 861, 871) was not found in the new_term_collector, but (859, 862, 861, 871) has the same atoms and terms, so (859, 862, 861, 871) will be removed from the new term collector
INFO:relative:hybrid_index_pair (860, 861, 862, 872) was not found in the new_term_collector, but (861, 860, 862, 872) has the same 

INFO:relative:hybrid_index_pair (2845, 2846, 2847, 2857) was not found in the new_term_collector, but (2846, 2845, 2847, 2857) has the same atoms and terms, so (2846, 2845, 2847, 2857) will be removed from the new term collector
INFO:relative:hybrid_index_pair (2901, 2904, 2902, 2912) was not found in the new_term_collector, but (2904, 2901, 2902, 2912) has the same atoms and terms, so (2904, 2901, 2902, 2912) will be removed from the new term collector
INFO:relative:hybrid_index_pair (3004, 3001, 3002, 3012) was not found in the new_term_collector, but (3001, 3004, 3002, 3012) has the same atoms and terms, so (3001, 3004, 3002, 3012) will be removed from the new term collector
INFO:relative:hybrid_index_pair (3023, 3033, 3025, 3032) was not found in the new_term_collector, but (3023, 3032, 3025, 3033) has the same atoms and terms, so (3023, 3032, 3025, 3033) will be removed from the new term collector
INFO:relative:hybrid_index_pair (3053, 3056, 3054, 3063) was not found in the new_te

In [21]:
pickle.dump(solvent_delivery.get_apo_htf(), open("T42A_apo.pickle", "wb" ))
pickle.dump(solvent_delivery.get_complex_htf(), open("T42A_complex.pickle", "wb" ))


#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [22]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [23]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 189.69678123582867), ('HarmonicAngleForce', 791.7783222646883), ('PeriodicTorsionForce', 3830.526380065008), ('NonbondedForce', -25197.606629558984), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 189.69678123582867), ('CustomAngleForce', 977.390455704673), ('CustomTorsionForce', 3830.526609908918), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [24]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [25]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 189.2884843576615), ('HarmonicAngleForce', 975.7683761903751), ('PeriodicTorsionForce', 3824.0195367640554), ('NonbondedForce', -24806.58521520547), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 189.2884843576615), ('CustomAngleForce', 977.390455704673), ('CustomTorsionForce', 3830.526609908918), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


In [32]:
scale_region = list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

### Test one alchemical region, no scale regions

In [26]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor

In [33]:
from __future__ import absolute_import

from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol, oechem
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory, RxnHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
    """
    Simple, stripped-down class to create a protein-ligand system and allow a mutation of a protein.
    this will allow support for the creation of _two_ relative free energy calculations:
        1. 'wildtype' - 'point mutant' complex hybrid.
        2. 'wildtype' - 'point mutant' protein hybrid (i.e. with ligand of interest unbound)
    Example (create full point mutation executor and run parallel tempering on both complex and apo phases):
        from pkg_resources import resource_filename
        protein_path = 'data/perses_jacs_systems/thrombin/Thrombin_protein.pdb'
        ligands_path = 'data/perses_jacs_systems/thrombin/Thrombin_ligands.sdf'
        protein_filename = resource_filename('openmmforcefields', protein_path)
        ligand_input = resource_filename('openmmforcefields', ligands_path)
        pm_delivery = PointMutationExecutor(protein_filename=protein_filename,
                                    mutation_chain_id='2',
                                    mutation_residue_id='198',
                                     proposed_residue='THR',
                                     phase='complex',
                                     conduct_endstate_validation=False,
                                     ligand_input=ligand_input,
                                     ligand_index=0,
                                     forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                                     barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                                     forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                                     small_molecule_forcefields='gaff-2.11')
        complex_htf = pm_delivery.get_complex_htf()
        apo_htf = pm_delivery.get_apo_htf()
        # Now we can build the hybrid repex samplers
        from perses.annihilation.lambda_protocol import LambdaProtocol
        from openmmtools.multistate import MultiStateReporter
        from perses.samplers.multistate import HybridRepexSampler
        from openmmtools import mcmc
        suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000
        for htf in [complex_htf, apo_htf]:
            lambda_protocol = LambdaProtocol(functions='default')
            reporter_file = 'reporter.nc'
            reporter = MultiStateReporter(reporter_file, analysis_particle_indices = htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
            hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                  collision_rate=5.0 / unit.picosecond,
                                                                                  n_steps=250,
                                                                                  reassign_velocities=False,
                                                                                  n_restart_attempts=20,
                                                                                  splitting="V R R R O R R R V",
                                                                                  constraint_tolerance=1e-06),
                                                                                  hybrid_factory=htf, online_analysis_interval=10)
            hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file=reporter, lambda_protocol=lambda_protocol, endstates=False)
            hss.extend(n_cycles)
    """
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 repartitioned_endstate=None,
                 rxn_field = True,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_file : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            repartitioned_endstate : int, default None
                the endstate (0 or 1) at which to build the RepartitionedHybridTopologyFactory. By default, this is None,
                meaning a vanilla HybridTopologyFactory will be built.
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_input, index=ligand_index)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for (top, pos, sys) in inputs:
            point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                                 system_generator=self.system_generator,
                                                                 chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                 max_point_mutants=1,
                                                                 residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                 allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                 aggregate=True) # Always allow aggregation

            topology_proposal = point_mutation_engine.propose(sys, top)

            # Only validate energy bookkeeping if the WT and proposed residues do not involve rings
            old_res = [res for res in top.residues() if res.id == mutation_residue_id][0]
            validate_bool = False if old_res.name in ring_amino_acids or proposed_residue in ring_amino_acids else True
            new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                                   validate_energy_bookkeeping=validate_bool)
            logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
                                                        validate_energy_bookkeeping=validate_bool)

            if rxn_field:
                factory = RxnHybridTopologyFactory
            else:
                if repartitioned_endstate is None:
                    factory = HybridTopologyFactory
                elif repartitioned_endstate in [0, 1]:
                    factory = RepartitionedHybridTopologyFactory

            forward_htf = factory(topology_proposal=topology_proposal,
                                  current_positions=pos,
                                  new_positions=new_positions,
                                  use_dispersion_correction=False,
                                  functions=None,
                                  softcore_alpha=None,
                                  bond_softening_constant=1.0,
                                  angle_softening_constant=1.0,
                                  soften_only_new=False,
                                  neglected_new_angle_terms=[],
                                  neglected_old_angle_terms=[],
                                  softcore_LJ_v2=True,
                                  softcore_electrostatics=True,
                                  softcore_LJ_v2_alpha=0.85,
                                  softcore_electrostatics_alpha=0.3,
                                  softcore_sigma_Q=1.0,
                                  interpolate_old_and_new_14s=flatten_exceptions,
                                  omitted_terms=None,
                                  endstate=repartitioned_endstate,
                                  flatten_torsions=flatten_torsions,
                                  scale_regions=[scale_region])

            if not topology_proposal.unique_new_atoms:
                assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
                assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            else:
                added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

            if not topology_proposal.unique_old_atoms:
                assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
                assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
                subtracted_valence_energy = 0.0
            else:
                subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


            if conduct_endstate_validation and repartitioned_endstate is None:
                zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta=beta, ENERGY_THRESHOLD=ENERGY_THRESHOLD)
                if zero_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 0 state is above the threshold ({ENERGY_THRESHOLD}): {zero_state_error}")
                if one_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 1 state is above the threshold ({ENERGY_THRESHOLD}): {one_state_error}")
            else:
                pass

            htfs.append(forward_htf)

        self.apo_htf = htfs[0]
        self.complex_htf = htfs[1] if ligand_input else None

In [34]:
solvent_delivery = PointMutationExecutor2("../../input/mmc2_barstar.pdb",
                        '1',
                        "42",
                        "ALA",
                        ligand_input="../../input/mmc2_barnase.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        conduct_endstate_validation=False,
                        rxn_field=True
                       )

INFO:root:solvating at 0.05 M using tip3p
INFO:root:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:{7: 11}
INFO:proposal_generator:{8: 11}
INFO:proposal_generator:{9: 11}
INFO:proposal_generator:{7: 12}
INFO:proposal_generator:{8: 12}
INFO:proposal_generator:{9: 12}
INFO:proposal_generator:{7: 13}
INFO:proposal_generator:{8: 13}
INFO:proposal_generator:{9: 13}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:pr

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 4 new atoms
INFO:geometry:	reduced angle potential = 1.2762201135827598.
INFO:geometry:	reduced angle potential = 0.11025002276606474.
INFO:geometry:	reduced angle potential = 0.1515221431721368.
INFO:geometry:	reduced angle potential = 0.26645108963779207.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 733 bond forces in the no-nonbonded final system
INFO:geometry:	there are 2628 angle forces in the no-nonbonded final system
INFO:geometry:	there are 4939 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

added energy components: [('CustomBondForce', 0.4083956553523612), ('CustomAngleForce', 1.622079514297996), ('CustomTorsionForce', 17.50313698245956), ('CustomBondForce', -81.92370902708515)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanomete

( nonscale_lambda_bonds * nonscale_region+scale_lambda_0_bonds * scale_region_0+interscale_lambda_0_bonds * interscale_region_0 ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


INFO:relative:hybrid_index_pair (700, 703, 702, 712) was not found in the new_term_collector, but (703, 700, 702, 712) has the same atoms and terms, so (703, 700, 702, 712) will be removed from the new term collector
INFO:relative:hybrid_index_pair (701, 702, 703, 713) was not found in the new_term_collector, but (702, 701, 703, 713) has the same atoms and terms, so (702, 701, 703, 713) will be removed from the new term collector
INFO:relative:hybrid_index_pair (750, 754, 752, 762) was not found in the new_term_collector, but (754, 750, 752, 762) has the same atoms and terms, so (754, 750, 752, 762) will be removed from the new term collector
INFO:relative:hybrid_index_pair (862, 859, 861, 871) was not found in the new_term_collector, but (859, 862, 861, 871) has the same atoms and terms, so (859, 862, 861, 871) will be removed from the new term collector
INFO:relative:hybrid_index_pair (860, 861, 862, 872) was not found in the new_term_collector, but (861, 860, 862, 872) has the same 

INFO:relative:hybrid_index_pair (2845, 2846, 2847, 2857) was not found in the new_term_collector, but (2846, 2845, 2847, 2857) has the same atoms and terms, so (2846, 2845, 2847, 2857) will be removed from the new term collector
INFO:relative:hybrid_index_pair (2901, 2904, 2902, 2912) was not found in the new_term_collector, but (2904, 2901, 2902, 2912) has the same atoms and terms, so (2904, 2901, 2902, 2912) will be removed from the new term collector
INFO:relative:hybrid_index_pair (3004, 3001, 3002, 3012) was not found in the new_term_collector, but (3001, 3004, 3002, 3012) has the same atoms and terms, so (3001, 3004, 3002, 3012) will be removed from the new term collector
INFO:relative:hybrid_index_pair (3023, 3033, 3025, 3032) was not found in the new_term_collector, but (3023, 3032, 3025, 3033) has the same atoms and terms, so (3023, 3032, 3025, 3033) will be removed from the new term collector
INFO:relative:hybrid_index_pair (3053, 3056, 3054, 3063) was not found in the new_te

In [35]:
pickle.dump(solvent_delivery.get_apo_htf(), open("T42A_apo_scale_region.pickle", "wb" ))
pickle.dump(solvent_delivery.get_complex_htf(), open("T42A_complex_scale_region.pickle", "wb" ))


#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [37]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [38]:
test_bond_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 189.69678123582867), ('HarmonicAngleForce', 791.7783222646883), ('PeriodicTorsionForce', 3830.526380065008), ('NonbondedForce', -25198.30731209033), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 189.69678123582867), ('CustomAngleForce', 995.6552423965976), ('CustomTorsionForce', 3830.526609908918), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [40]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [41]:
test_bond_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 189.99812484509627), ('HarmonicAngleForce', 994.0331628822997), ('PeriodicTorsionForce', 3824.01891675258), ('NonbondedForce', -24572.84582355946), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 189.9981248450963), ('CustomAngleForce', 995.6552423965976), ('CustomTorsionForce', 3830.526609908918), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom bond force and standard bond force energies are equal!


In [ ]:
# TO DO: actually test the scaling of the energies matches